# Task 4 — VQE

In [1]:
import numpy as np

In [2]:
H = np.array([[1, 0, 0, 0],
              [0, 0, -1, 0],
              [0, -1, 0, 0],
              [0, 0, 0, 1]])

To find the lowest Eigenvalue of this matrix $H$, I will start by decomposing it into a sum of Pauli terms ($II$, $XX$, $YY$ and $ZZ$).

First let's define all of the one-qubit Pauli matrices:

In [3]:
I = np.array([[1, 0],
              [0, 1]])
X = np.array([[0, 1],
              [1, 0]])
Y = np.array([[0, -1j],
              [1j, 0]])
Z = np.array([[1, 0],
              [0, -1]])

And $H$ can be decomposed into a sum of $II$, $XX$, $YY$, and $ZZ$, so let's define those matrices:

In [4]:
II = np.kron(I, I)
XX = np.kron(X, X)
YY = np.kron(Y, Y)
ZZ = np.kron(Z, Z)

In [5]:
print(II)
print(XX)
print(YY)
print(ZZ)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[[0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]
[[ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]
[[ 1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]]


We can hence see that the span of these matrices is given by:

$\text{span}(II, XX, YY, ZZ) = \left\{
\begin{pmatrix}
a & 0 & 0 & d\\
0 & b & c & 0\\
0 & c & b & 0\\
d & 0 & 0 & a\\
\end{pmatrix}
\, \middle| \, a, b, c, d \in \mathbb{C}\right\}$

$\begin{pmatrix}
a & 0 & 0 & d\\
0 & b & c & 0\\
0 & c & b & 0\\
d & 0 & 0 & a\\
\end{pmatrix}= \frac{a}{2}(II + ZZ) + \frac{b}{2}(II - ZZ) + \frac{c}{2}(XX + YY) + \frac{d}{2}(XX - YY)$

$= \frac{1}{2} ((a + b)II + (c + d) XX + (c - d) YY + (a - b) ZZ)$

and so we can decompose $H$ this way, as it is of this form.

Let's define a function to do just that:

In [6]:
def decompose(matrix):
    """
    Takes a 4x4 matrix (represented as a numpy array), and tries to
    decompose it into a sum of II, XX, YY, ZZ. Returns a tuple
    (w, x, y, z) such that w II + x XX + y YY + z ZZ is the matrix.
    If such a decomposition is not possible, it will raise an
    AssertionError.
    """
    a = matrix[0, 0]
    b = matrix[1, 1]
    c = matrix[2, 1]
    d = matrix[3, 0]
    w = (a + b) / 2
    x = (c + d) / 2
    y = (c - d) / 2
    z = (a - b) / 2
    assert(np.all(w * II + x * XX + y * YY + z * ZZ == matrix))
    return (w, x, y, z)

w, x, y, z = decompose(H)
print(w, x, y, z)

0.5 -0.5 -0.5 0.5


Hence we can rewrite H as

$H = \frac{1}{2}(II - XX - YY + ZZ)$

which checks out as

$\frac{1}{2}(II - XX - YY + ZZ) = \frac{1}{2}\left(
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
\end{pmatrix} - \begin{pmatrix}
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
1 & 0 & 0 & 0\\
\end{pmatrix} - \begin{pmatrix}
0 & 0 & 0 & -1\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
-1 & 0 & 0 & 0\\
\end{pmatrix} + \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & 0 & 0 & 1\\
\end{pmatrix}\right)
$

$=\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & 0 & 1\\
\end{pmatrix}$

$=H$

Now we can implement this circuit in Qiskit:

In [7]:
from qiskit import QuantumCircuit, Aer, execute

First let's write a function to create a state to represent our ansatz.

For our ansatz we are going to use $\left\vert\psi\right\rangle = (R_X(\theta)\otimes I)(CX_{01})(H\otimes I) \left\vert00\right\rangle$, where $R_X(\theta)$ is a X-rotation gate paramaterized by an angle $\theta$, and $CX_{01}$ is a controlled-not gate controlled on qubit-0 acting on qubit-1, and so the only parameter in our ansatz is $\theta$.

This circuit looks like:
```
     ┌───┐     ┌───────────┐
|0>: ┤ H ├──■──┤ RX(theta) ├
     └───┘┌─┴─┐└───────────┘
|0>: ─────┤ X ├─────────────
          └───┘          
```

In [8]:
def prepare_ansatz_circuit(theta):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    qc.rx(theta, 0)
    return qc

We can then write a function to measure a circuit to apply the two-qubit Pauli measurements $XX$, $YY$, or $ZZ$ to a given circuit:

In [9]:
def measure_circuit(operator, qc):
    qc = qc.copy()
    assert(operator in ["XX", "YY", "ZZ"])
    if operator == "XX":
        qc.h([0, 1])
    elif operator == "YY":
        qc.sdg([0, 1])
        qc.h([0, 1])
    qc.measure_all()
    return qc

We can then use this to write a function to estimate the expectation of a two-qubit Pauli operator or the identity:

In [10]:
simulator = Aer.get_backend('qasm_simulator')
def measure_pauli_expectation(qc, operator, shots):
    if operator == "II":
        return 1
    else:
        measurement = measure_circuit(operator, qc)
        result = execute(measurement, backend = simulator, shots = shots).result()
        counts = result.get_counts(measurement)
        positives = sum([counts[m] for m in counts if m == "00" or m == "11"])
        return 2 * positives / shots - 1

And we can then use this to write another function to estimate the expectation of any linear combination of these operators $H = w II + x XX + y YY + z ZZ$:

In [11]:
def measure_energy_expectation(qc, w, x, y, z, n):
    EII = measure_pauli_expectation(qc, "II", n)
    EXX = measure_pauli_expectation(qc, "XX", n)
    EYY = measure_pauli_expectation(qc, "YY", n)
    EZZ = measure_pauli_expectation(qc, "ZZ", n)
    return w * EII + x * EXX + y * EYY + z * EZZ

Now we have defined all the necessary functions to hopefully find the minimum eigenvalue.

Now let's try a wide variety of ansätze by trying different values of our parameter $\theta$. We only need to try angles in the range $[0, 2\pi)$, as an $R_X(\theta)$ gate is periodic with order $2\pi$.

In [12]:
thetas = np.linspace(0, 2 * np.pi, 65)

We can then evaluate $\left\langle H \right\rangle$ for all these ansätze — by the variational principle we know that $\left\langle H \right\rangle \geq E_{\text{min}}$ (the minimum eigenvalue), and the minimum value of $\left\langle H \right\rangle$ should hopefully approximate the minimum eigenvalue (assuming its eigenstate is one of the ansätze — if not, we will only get an approximate upper bound on the eigenvalue).

In [13]:
minE = 1
for theta in thetas:
    qc = prepare_ansatz_circuit(theta)
    E = measure_energy_expectation(qc, w, x, y, z, 1000)
    if E < minE:
        minTheta = theta
        minE = E
print("The minimum eigenvalue seems to be", minE, "which comes from theta =", minTheta)

The minimum eigenvalue seems to be -1.0 which comes from theta = 3.141592653589793


Assuming all of my code is working correctly, this should give an eigenvalue of $E_\text{min} \approx -1$, with $\theta \approx \pi$.

This state is

$\left\vert\psi\right\rangle = (R_X(\pi)\otimes I)(CX_{01})(H\otimes I) \left\vert00\right\rangle$
$= \left(\begin{pmatrix}
\cos(\pi/2) & -i\sin(\pi/2)\\
-i\sin(\pi/2) & \cos(\pi/2)\\
\end{pmatrix}\otimes\begin{pmatrix}
1 & 0\\
0 & 1\\
\end{pmatrix}\right)\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
\end{pmatrix}\left(\frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1 & -1\\
\end{pmatrix}\otimes\begin{pmatrix}
1 & 0\\
0 & 1\\
\end{pmatrix}\right) \begin{pmatrix}
1\\
0\\
0\\
0\\
\end{pmatrix}$
$= \frac{1}{\sqrt{2}}\begin{pmatrix}
0 & 0 & -i & 0\\
0 & 0 & 0 & -i\\
-i & 0 & 0 & 0\\
0 & -i & 0 & 0\\
\end{pmatrix}\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
\end{pmatrix}\begin{pmatrix}
1 & 0 & 1 & 0\\
0 & 1 & 0 & 1\\
1 & 0 & -1 & 0\\
0 & 1 & 0 & -1\\
\end{pmatrix} \begin{pmatrix}
1\\
0\\
0\\
0\\
\end{pmatrix}$
$= \frac{1}{\sqrt{2}}\begin{pmatrix}
0 & 0 & -i & 0\\
0 & 0 & 0 & -i\\
-i & 0 & 0 & 0\\
0 & -i & 0 & 0\\
\end{pmatrix}\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
\end{pmatrix}\begin{pmatrix}
1\\
0\\
1\\
0\\
\end{pmatrix}$
$=\frac{1}{\sqrt{2}}\begin{pmatrix}
0 & 0 & -i & 0\\
0 & 0 & 0 & -i\\
-i & 0 & 0 & 0\\
0 & -i & 0 & 0\\
\end{pmatrix}\begin{pmatrix}
1\\
0\\
0\\
1\\
\end{pmatrix}$
$=\frac{1}{\sqrt{2}}\begin{pmatrix}
0\\
-i\\
-i\\
0\\
\end{pmatrix}$
$=-\frac{i}{\sqrt{2}}\begin{pmatrix}
0\\
1\\
1\\
0\\
\end{pmatrix}$

So our state is $|\psi\rangle=\frac{1}{\sqrt{2}}(\left\vert01\right\rangle + \left\vert10\right\rangle) = \frac{1}{\sqrt{2}}\begin{pmatrix}
0\\
1\\
1\\
0\\
\end{pmatrix}$ up to a global phase factor.

We can see that this is an eigenvector as 

$H \left\vert\psi\right\rangle = \frac{1}{\sqrt{2}} \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & 0 & 1\\
\end{pmatrix} \begin{pmatrix}
0\\
1\\
1\\
0\\
\end{pmatrix}= \frac{1}{\sqrt{2}}\begin{pmatrix}
0\\
-1\\
-1\\
0\\
\end{pmatrix} = - \left\vert\psi\right\rangle$

and its corresponding eigenvalue is hence indeed -1.

To check that this is the minimum eigenvalue, we can find the eigenvalues by finding the roots of the characterstic polynomial:

$p_H(\lambda) = \det(\lambda I - H) = \det\left(\begin{pmatrix}
\lambda - 1 & 0 & 0 & 0\\
0 & \lambda & -1 & 0\\
0 & -1 & \lambda & 0\\
0 & 0 & 0 & \lambda - 1\\
\end{pmatrix}\right) = (\lambda - 1)^2 \det\left(\begin{pmatrix}
\lambda & -1\\
-1 & \lambda &\\
\end{pmatrix}\right) = (\lambda - 1)^2 (\lambda^2 - 1) = (\lambda - 1)^3 (\lambda + 1)$

So the only eigenvalues are $\pm 1$, and so we have found the minimum eigenvalue.

If we wanted to make this code better, then we could decompose it into a wider range of two-qubit Pauli gates (e.g. XY, IZ etc.) so that it can work on a wider range of input matrices.

Another thing that could help would be using a more flexible ansatz to allow us to explore more possible states. In doing so, we would probably need to use an optimizer like Stochastic Gradient Descent, as checking every possible state would be very time-inefficient.